In [24]:
# As usual, a bit of setup

import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.cnn import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from cs231n.layers import *
from cs231n.fast_layers import *
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [26]:
x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num = np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num = np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num = 3)

conv_param = {'stride': 2, 'pad': 1}

In [27]:
## Unpacking the convolutional parameters.
stride, padWidth = conv_param['stride'], conv_param['pad']

## Padding the input with zeroes.
xPadded = np.pad(x, pad_width = ((0, 0), (0, 0), (padWidth, padWidth), (padWidth, padWidth)), mode = 'constant', constant_values = 0)
xPadded.shape

(2, 3, 6, 6)

In [28]:
## Defining the input height, width, depth and inputSize.
inputHeight = x.shape[-2]
inputWidth = x.shape[-1]
inputDepth = x.shape[1]
inputSize = x.shape[0]

## Defining the filter height, width, depth and number of filters.
filterHeight = w.shape[-2]
filterWidth = w.shape[-1]
filerDepth = w.shape[1]
numFilters = w.shape[0]

## Defining the output height, width and depth.
outputHeight = ((inputHeight - filterHeight + 2 * padWidth) / stride + 1)
outputWidth = ((inputWidth - filterWidth + 2 * padWidth) / stride + 1)
outputDepth = numFilters

## Initializing the output activation map.
outputActivationMap = np.empty([inputSize, outputDepth, outputWidth, outputHeight])
print outputActivationMap.shape

(2, 3, 2, 2)


In [31]:
## Obtaining the necessary input slices over which
## the weight matrices will convolve.

for l in range(0, inputSize):

    for k in range(0, numFilters):

        for i in range(0, outputHeight):

            for j in range(0, outputWidth):

                ## Obtaining the input slice.
                xImageSlice = xPadded[l, :, i * stride : i * stride + filterHeight, j * stride : j * stride + filterWidth]
                
                ## Performing the dot product of the weight matrix with the image slice.
                outputActivationMap[l, k, i, j] = np.sum(xImageSlice * w[k]) + b[k]